# Test tokens

In [9]:
import re

In [2]:
!ls \test

short_test.tex test.tex


In [3]:
filename = './test/short_test.tex'

In [7]:
with open(filename) as f:
    raw_text = f.read()

In [8]:
raw_text

'Some text here. Another sentence.\nAnother line.\n\\remove{This will be deleted.}\nThis will not. \n\\add{This will be added.} But be careful with equations\n\\begin{equation}\n    \\text{Ra} = \\frac{\\alpha g(r_\\text{cmb})\\Delta T L}{\\nu \\Omega},\n\\end{equation}\n\nwhich can be tricky. \\add{Also take care with citations \\cite{einstein1905} and references \\eqref{eq:modes}.}\n\\change{Finally, change commands need to be treated correctly.}{As a last point, correct changes should be made.}\n'

In [25]:
add_start_pos = [m.start() for m in re.finditer('\\\\add', raw_text)]
add_start_pos

[95, 274]

In [26]:
remove_start_pos = [m.start() for m in re.finditer('\\\\remove', raw_text)]
remove_start_pos

[48]

In [27]:
change_start_pos = [m.start() for m in re.finditer('\\\\change', raw_text)]
change_start_pos

[363]

In [95]:
def add_command(text):
    add_start_pos = [m.start() for m in re.finditer('\\\\add', text)]
    add_end_pos = len(add_start_pos)*[None]
    add_strings = len(add_start_pos)*[None]
    n = len(text)
    
    for (i,v) in enumerate(add_start_pos):
        add_end_pos[i], add_strings[i] = find_interior(i,v,5,text)
    
    new_text = '%s' % text
    for i in reversed(range(len(add_start_pos))):
        print(text[add_start_pos[i]:add_end_pos[i]])
        print(add_strings[i])
        print('\n')
        new_text = new_text[:add_start_pos[i]] + add_strings[i] + new_text[add_end_pos[i]:]
    return new_text
def remove_command(text):
    remove_start_pos = [m.start() for m in re.finditer('\\\\remove', text)]
    remove_end_pos = len(remove_start_pos)*[None]
    remove_strings = len(remove_start_pos)*[None]
    n = len(text)
    
    for (i,v) in enumerate(remove_start_pos):
        remove_end_pos[i], remove_strings[i] = find_interior(i,v,8,text)
    
    new_text = '%s' % text
    for i in reversed(range(len(remove_start_pos))):
        print(text[remove_start_pos[i]:remove_end_pos[i]])
        print(remove_strings[i])
        print('\n')
        new_text = new_text[:remove_start_pos[i]] + new_text[remove_end_pos[i]:]
    return new_text
def change_command(text):
    change1_start_pos = [m.start() for m in re.finditer('\\\\change', text)]
    change1_end_pos = len(change1_start_pos)*[None]
    change1_strings = len(change1_start_pos)*[None]
    change2_start_pos = len(change1_start_pos)*[None]
    change2_end_pos = len(change1_start_pos)*[None]
    change2_strings = len(change1_start_pos)*[None]
    n = len(text)
    
    for (i,v) in enumerate(change1_start_pos):
        # First text
        change1_end_pos[i], change1_strings[i] = find_interior(i,v,8,text)
        # Second text
        change2_start_pos[i] = change1_end_pos[i]
        change2_end_pos[i], change2_strings[i] = find_interior(i,change2_start_pos[i],1,text)
    
    new_text = '%s' % text
    for i in reversed(range(len(change1_start_pos))):
        print(text[change1_start_pos[i]:change1_end_pos[i]])
        print(change1_strings[i])
        print(text[change2_start_pos[i]:change2_end_pos[i]])
        print(change2_strings[i])
        print('\n')
        new_text = new_text[:change1_start_pos[i]] + change2_strings[i] + new_text[change2_end_pos[i]:]
    return new_text
def find_interior(i,v,start,text):
    print(i,v,text[v:v+start])
    bracket_start_pos = v + start
    bracket_level = 1
    ind = bracket_start_pos

    #tmp_str = ''
    interior_string = ''
    while bracket_level > 0:
        if text[ind] == '{':
            bracket_level += 1
        elif text[ind] == '}':
            bracket_level -= 1
        #tmp_str += str(bracket_level)
        interior_string += text[ind]
        ind += 1
    #print(tmp_str)
    print(interior_string[:-1])
    print(ind)
    print('\n')
    
    return ind, interior_string[:-1]
def update_text(text):
    text = add_command(text)
    text = remove_command(text)
    text = change_command(text)
    return text
update_text(raw_text)

0 95 \add{
This will be added.
120


1 274 \add{
Also take care with citations \cite{einstein1905} and references \eqref{eq:modes}.
362


\add{Also take care with citations \cite{einstein1905} and references \eqref{eq:modes}.}
Also take care with citations \cite{einstein1905} and references \eqref{eq:modes}.


\add{This will be added.}
This will be added.


0 48 \remove{
This will be deleted.
78


\remove{This will be deleted.}
This will be deleted.


0 321 \change{
Finally, change commands need to be treated correctly.
384


0 384 {
As a last point, correct changes should be made.
434


\change{Finally, change commands need to be treated correctly.}
Finally, change commands need to be treated correctly.
{As a last point, correct changes should be made.}
As a last point, correct changes should be made.




'Some text here. Another sentence.\nAnother line.\n\nThis will not. \nThis will be added. But be careful with equations\n\\begin{equation}\n    \\text{Ra} = \\frac{\\alpha g(r_\\text{cmb})\\Delta T L}{\\nu \\Omega},\n\\end{equation}\n\nwhich can be tricky. Also take care with citations \\cite{einstein1905} and references \\eqref{eq:modes}.\nAs a last point, correct changes should be made.\n'

In [96]:
raw_text

'Some text here. Another sentence.\nAnother line.\n\\remove{This will be deleted.}\nThis will not. \n\\add{This will be added.} But be careful with equations\n\\begin{equation}\n    \\text{Ra} = \\frac{\\alpha g(r_\\text{cmb})\\Delta T L}{\\nu \\Omega},\n\\end{equation}\n\nwhich can be tricky. \\add{Also take care with citations \\cite{einstein1905} and references \\eqref{eq:modes}.}\n\\change{Finally, change commands need to be treated correctly.}{As a last point, correct changes should be made.}\n'

In [83]:
remove_command(raw_text)

477
0 48 \remove{
This will be deleted.
78


\remove{This will be deleted.}
This will be deleted.




'Some text here. Another sentence.\nAnother line.\n\nThis will not. \n\\add{This will be added.} But be careful with equations\n\\begin{equation}\n    \\text{Ra} = \\frac{\\alpha g(r_\\text{cmb})\\Delta T L}{\\nu \\Omega},\n\\end{equation}\n\nwhich can be tricky. \\add{Also take care with citations \\cite{einstein1905} and references \\eqref{eq:modes}.}\n\\change{Finally, change commands need to be treated correctly.}{As a last point, correct changes should be made.}\n'

In [80]:
raw_text

'Some text here. Another sentence.\nAnother line.\n\\remove{This will be deleted.}\nThis will not. \n\\add{This will be added.} But be careful with equations\n\\begin{equation}\n    \\text{Ra} = \\frac{\\alpha g(r_\\text{cmb})\\Delta T L}{\\nu \\Omega},\n\\end{equation}\n\nwhich can be tricky. \\add{Also take care with citations \\cite{einstein1905} and references \\eqref{eq:modes}.}\n\\change{Finally, change commands need to be treated correctly.}{As a last point, correct changes should be made.}\n'

In [97]:
def add_command(text):
    add_start_pos = [m.start() for m in re.finditer('\\\\add', text)]
    add_end_pos = len(add_start_pos)*[None]
    add_strings = len(add_start_pos)*[None]
    n = len(text)
    
    for (i,v) in enumerate(add_start_pos):
        add_end_pos[i], add_strings[i] = find_interior(i,v,5,text)
    
    new_text = '%s' % text
    for i in reversed(range(len(add_start_pos))):
        new_text = new_text[:add_start_pos[i]] + add_strings[i] + new_text[add_end_pos[i]:]
    return new_text

def remove_command(text):
    remove_start_pos = [m.start() for m in re.finditer('\\\\remove', text)]
    remove_end_pos = len(remove_start_pos)*[None]
    remove_strings = len(remove_start_pos)*[None]
    n = len(text)
    
    for (i,v) in enumerate(remove_start_pos):
        remove_end_pos[i], remove_strings[i] = find_interior(i,v,8,text)
    
    new_text = '%s' % text
    for i in reversed(range(len(remove_start_pos))):
        new_text = new_text[:remove_start_pos[i]] + new_text[remove_end_pos[i]:]
    return new_text

def change_command(text):
    change1_start_pos = [m.start() for m in re.finditer('\\\\change', text)]
    change1_end_pos = len(change1_start_pos)*[None]
    change1_strings = len(change1_start_pos)*[None]
    change2_start_pos = len(change1_start_pos)*[None]
    change2_end_pos = len(change1_start_pos)*[None]
    change2_strings = len(change1_start_pos)*[None]
    n = len(text)
    
    for (i,v) in enumerate(change1_start_pos):
        # First text
        change1_end_pos[i], change1_strings[i] = find_interior(i,v,8,text)
        # Second text
        change2_start_pos[i] = change1_end_pos[i]
        change2_end_pos[i], change2_strings[i] = find_interior(i,change2_start_pos[i],1,text)
    
    new_text = '%s' % text
    for i in reversed(range(len(change1_start_pos))):
        new_text = new_text[:change1_start_pos[i]] + change2_strings[i] + new_text[change2_end_pos[i]:]
    return new_text

def find_interior(i,v,start,text):
    bracket_start_pos = v + start
    bracket_level = 1
    ind = bracket_start_pos

    interior_string = ''
    while bracket_level > 0:
        if text[ind] == '{':
            bracket_level += 1
        elif text[ind] == '}':
            bracket_level -= 1
        interior_string += text[ind]
        ind += 1
    
    return ind, interior_string[:-1]
def update_text(text):
    text = add_command(text)
    text = remove_command(text)
    text = change_command(text)
    return text
update_text(raw_text)

'Some text here. Another sentence.\nAnother line.\n\nThis will not. \nThis will be added. But be careful with equations\n\\begin{equation}\n    \\text{Ra} = \\frac{\\alpha g(r_\\text{cmb})\\Delta T L}{\\nu \\Omega},\n\\end{equation}\n\nwhich can be tricky. Also take care with citations \\cite{einstein1905} and references \\eqref{eq:modes}.\nAs a last point, correct changes should be made.\n'

In [98]:
raw_text

'Some text here. Another sentence.\nAnother line.\n\\remove{This will be deleted.}\nThis will not. \n\\add{This will be added.} But be careful with equations\n\\begin{equation}\n    \\text{Ra} = \\frac{\\alpha g(r_\\text{cmb})\\Delta T L}{\\nu \\Omega},\n\\end{equation}\n\nwhich can be tricky. \\add{Also take care with citations \\cite{einstein1905} and references \\eqref{eq:modes}.}\n\\change{Finally, change commands need to be treated correctly.}{As a last point, correct changes should be made.}\n'